In [2]:
import openai
from importlib import reload
from llm_client import llm_client
from llm_client.agent import MultiTurnAgent
from llm_client.prompt_builder import  PromptBuilder
from llm_client.document_vector_store import (
    EmbeddingProcessor,
    Document,
    DocumentStore,
    SimpleDocument,
    VectorStore
)
from pipelines.kme_doc import KMEDocument
from tqdm import tqdm
import pandas as pd
import os
from config.settings import Settings
from content_retrieval import query_document, llm,doc_store ,embed,vector_store
from pipelines.processors import summarize_new_documents, add_new_documents_to_docstore
from interface.project import Project
settings = Settings()

In [2]:
kme_vertaaltabel = pd.read_csv(settings.data_root /  "kme_vertaaltabel.csv",sep=';').set_index("KME_ID")

# Opbouwen Doc store van pdf

In [3]:
kme_documents = pd.read_pickle(settings.content_folder / "extracted_df_km.pkl").reset_index()
kme_documents = kme_documents.set_index("km_nummer").join(kme_vertaaltabel).reset_index()
ob_documents = kme_documents.query("BELASTINGSOORT == 'OB - Omzetbelasting'")
results = add_new_documents_to_docstore(ob_documents,doc_store)

In [4]:
summary_processor = PromptBuilder(template_path='prompt_templates',name='summarize')

### Samenvatten en segmenteren content

In [6]:
stats = summarize_new_documents(
    doc_store=doc_store,
    prompt_builder=summary_processor,
    llm=llm,
    max_workers=10,
    start=0,
    count=1500,
    show_progress=True
)